In [2]:
import pandas as pd


btc = pd.read_csv('/Users/hatim/Desktop/Applied Forecasting/Final Project/Algo Trading/btc_daily.csv', index_col=0, parse_dates=True)
btc['Risk Free']=0.03
btc

,Open,High,Low,Close,Volume,Risk Free
2015-07-20,277.98,280.00,277.37,280.00,782.883420,0.03
2015-07-21,279.96,281.27,276.85,277.32,4943.559434,0.03
2015-07-22,277.33,278.54,275.01,277.89,4687.909383,0.03
2015-07-23,277.96,279.75,276.28,277.39,5306.919575,0.03
2015-07-24,277.23,291.52,276.43,289.12,7362.469083,0.03
...,...,...,...,...,...,...
2025-04-04,83178.68,84720.67,81643.54,83860.16,14593.030929,0.03
2025-04-05,83859.78,84238.35,82346.61,83498.25,2836.429207,0.03
2025-04-06,83505.88,83773.58,77058.99,78370.75,11014.859477,0.03
2025-04-07,78370.15,81223.67,74420.69,79140.01,26706.529308,0.03


# Finading the optimal trading strategy in Hindsight
- 1 will be buy
- -1 will be sell
- 0 will be hold

The goal is to maximize return while considering trasaction costs. We ignore the volatility adjustment here because it then becomes a non-convex optimization problem that is computationally infeasible. Furthermore, in hindsight the problem is deterministic and the concept of risk is incoherent so we ignore it.


## Defining the optimization problem
### Starting assumptions
we will reduce the labels to the following:
- 1 will be in the market (buy all that we can)
- 0 will be out of the market (sell all that we can)
This can then be trasnformed into our original labels easily.

We use dynamic programming for this



In [3]:
import numpy as np
import plotly.graph_objects as go

class OptStrat:
    def __init__(self, df, transaction_cost=0.005):
        self.df = df
        self.prices = df['Close'].values
        self.rf_array = ((1+ df['Risk Free'])**(1/365) - 1).values
        self.transaction_cost = transaction_cost

        # get optimal strategy
        decisions, returns = self.optimize_trading_strategy()
        
        self.df['Signals'] = self.transform_signals(decisions)

        self.plot(returns)

    def optimize_trading_strategy(self):
        """
        Optimize a binary Bitcoin trading strategy using dynamic programming.
        
        Args:
            prices (list): Daily Bitcoin closing prices
            risk_free_rate (float): Daily risk-free rate (as a decimal)
            transaction_cost (float): Transaction cost rate (default: 0.0005 for 0.05%)
            
        Returns:
            tuple: (optimal_policy, expected_return)
        """
        prices = self.prices
        transaction_cost = self.transaction_cost
        risk_free_rate = self.rf_array


        T = len(prices)-1  # Number of trading days
        
        # Calculate Bitcoin daily returns
        btc_returns = [(prices[t+1] - prices[t])/prices[t] for t in range(T)]
        
        # Initialize memoization table and decisions
        memo = np.zeros((T+1, 2))
        decisions = np.zeros((T, 2), dtype=int)
        
        # Fill table bottom-up
        for t in range(T-1, -1, -1):
            for prev_action in [0, 1]:
                # Option 1: Bitcoin
                btc_return = btc_returns[t]
                if prev_action == 0:  # Switch to Bitcoin
                    value_bitcoin = btc_return - transaction_cost + memo[t+1, 1]
                else:  # Stay in Bitcoin
                    value_bitcoin = btc_return + memo[t+1, 1]
                
                # Option 2: Risk-free
                if prev_action == 1:  # Switch to risk-free
                    value_risk_free = risk_free_rate[t] - transaction_cost + memo[t+1, 0]
                else:  # Stay in risk-free
                    value_risk_free = risk_free_rate[t] + memo[t+1, 0]
                
                # Select better option
                if value_bitcoin > value_risk_free:
                    memo[t, prev_action] = value_bitcoin
                    decisions[t, prev_action] = 1
                else:
                    memo[t, prev_action] = value_risk_free
                    decisions[t, prev_action] = 0
        
        # Reconstruct optimal policy
        optimal_policy = []
        current_action = 0  # Assume we start with no position
        
        for t in range(T):
            current_action = decisions[t, current_action]
            optimal_policy.append(current_action)
        optimal_policy.append(np.nan)
        
        # Expected return from optimal policy
        expected_return = memo[0, 0]
        
        return optimal_policy, expected_return
    
    def transform_signals(self, decisions):
        """
        Transform binary decisions (0,1) into trading signals (-1,0,1)
        
        Args:
            decisions (array-like): Array of 0s and 1s representing out/in market
        
        Returns:
            array-like: Array of -1 (sell), 0 (hold), 1 (buy)
        """
        # Convert to numpy array if not already
        signals = np.array(decisions)
        
        # Get the differences between consecutive elements
        changes = np.diff(signals, prepend=0)
        
        # Initialize output array
        trading_signals = np.zeros_like(signals)
        
        # 1 when changing from 0 to 1 (buy signal)
        trading_signals[changes == 1] = 1
        
        # -1 when changing from 1 to 0 (sell signal)
        trading_signals[changes == -1] = -1
        
        return trading_signals
    
    def plot(self,returns):
        print(f"Return = {returns**100:.2}%")
        # Create figure
        fig = go.Figure()

        # Add base price line
        fig.add_trace(
            go.Scatter(
                x=btc.index,
                y=btc['Close'],
                name='BTC Price',
                line=dict(color='lightgrey'),
            )
        )

        # Add buy signals (green dots)
        buy_signals = btc[btc['Signals'] == 1]  
        fig.add_trace(
            go.Scatter(
                x=buy_signals.index,
                y=buy_signals['Close'],
                name='Buy Signal',
                mode='markers',
                marker=dict(
                    color='green',
                    size=2,
                    symbol='circle'
                )
            )
        )

        # Add sell signals (red dots)
        sell_signals = btc[btc['Signals'] == -1]
        fig.add_trace(
            go.Scatter(
                x=sell_signals.index,
                y=sell_signals['Close'],
                name='Sell Signal',
                mode='markers',
                marker=dict(
                    color='red',
                    size=2,
                    symbol='circle'
                )
            )
        )

        # Update layout with range slider
        fig.update_layout(
            title='BTC Price with Optimal Trading Signals',
            yaxis_title='Price',
            xaxis_title='Date',
            xaxis=dict(
                rangeslider=dict(visible=True),
                type='date'
            ),
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01
            )
        )

        fig.show()


strat=OptStrat(df=btc)

Return = 3.5e+159%


In [4]:
strat.df['Signals'].value_counts()

Signals
 0.0    2342
 1.0     605
-1.0     604
Name: count, dtype: int64

In [5]:
strat.df

,Open,High,Low,Close,Volume,Risk Free,Signals
2015-07-20,277.98,280.00,277.37,280.00,782.883420,0.03,0.0
2015-07-21,279.96,281.27,276.85,277.32,4943.559434,0.03,1.0
2015-07-22,277.33,278.54,275.01,277.89,4687.909383,0.03,0.0
2015-07-23,277.96,279.75,276.28,277.39,5306.919575,0.03,0.0
2015-07-24,277.23,291.52,276.43,289.12,7362.469083,0.03,0.0
...,...,...,...,...,...,...,...
2025-04-04,83178.68,84720.67,81643.54,83860.16,14593.030929,0.03,-1.0
2025-04-05,83859.78,84238.35,82346.61,83498.25,2836.429207,0.03,0.0
2025-04-06,83505.88,83773.58,77058.99,78370.75,11014.859477,0.03,1.0
2025-04-07,78370.15,81223.67,74420.69,79140.01,26706.529308,0.03,0.0


# Learning signals and predicting for the future

## Feature Engineering